### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [14]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown
import os

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [15]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [16]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [17]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [18]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [19]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
                                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                                        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                                        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                                                        model=os.getenv("AZURE_OPENAI_MODEL"))
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
                                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                                        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                                        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                                                        model=os.getenv("AZURE_OPENAI_MODEL"))
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = AzureOpenAIChatCompletionClient(azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),
                                                        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                                        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                                                        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
                                                        model=os.getenv("AZURE_OPENAI_MODEL"))
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [20]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [21]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [26]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some pros of using AutoGen for your AI Agent project:

1. **Scalability**: AutoGen's modularity and extensibility allow for the creation of scalable systems, making it easier to adapt to growing requirements.

2. **Ease of Use**: It comes with integrated observability and debugging tools that simplify the monitoring and control of agent workflows, enhancing the overall development experience.

These features can significantly streamline the development process and improve the efficiency of managing multiple agents in complex systems.

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen in an AI Agent project:

1. **Dependency on Input Quality**: The effectiveness of AutoGen is highly dependent on the quality of the input data and parameters set by the user. Poorly defined inputs can lead to suboptimal or inaccurate outputs, necessitating careful selection and validation.

2. **Limited Customization**: While AutoGen can automate content generation, it may not provide the level of customization that some projects require. Users might find it challenging to tailor the output specifically to their needs.

3. **Complexity in Integration**: Incorporating AutoGen into existing systems can be complex depending on the architecture and requirements of the project, potentially leading to increased development time and costs.

4. **Learning Curve**: Utilizing AutoGen may require a learning curve, especially for users not familiar with its functionalities or the underlying technology, which could delay project timelines.

5. **Over-reliance on Automation**: Heavy reliance on automated systems like AutoGen could lead to a decrease in human oversight, risking the propagation of errors or inappropriate content if not monitored rigorously.

6. **Performance Variability**: The consistency and quality of outputs may vary, as it might not always perform optimally across all tasks or types of content, leading to unpredictable results.

7. **Ethical Concerns**: There are potential ethical issues regarding bias and the accuracy of generated content, which may impact decision-making in sensitive applications.

These points should be considered critically while deciding whether AutoGen is suitable for your AI Agent project. 

TERMINATE



## Decision:

Based on the research from the team, I recommend against using AutoGen for the project. While its scalability and ease of use are attractive, the cons highlight significant concerns such as dependency on input quality, limited customization, and integration complexity. Additionally, the potential for performance variability and ethical concerns could pose risks to the project's integrity. Taking these factors into account, a different solution may provide a more reliable and tailored approach for the AI Agent project.

TERMINATE

In [27]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

RuntimeError: Runtime is not running.

In [28]:
await host.stop()

RuntimeError: Host runtime is not started.